In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pyproj
from vis_tools import grab_GMT_features, read_HYSPLIT_netCDF

###############################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '/opt/hysplit/18032106_ngauruhoe_12.0_0.01.nc'

# SPECIFY: model time step
TIME_STEP = 0

# SPECIFY: plotting params
ASH_MIN = 10**-2
ASH_MAX = 10**3
#NUM_CON = 6
RES = 'f'
###############################################

# projection stuff
WGS84 = pyproj.Proj('+init=epsg:4326')
NZTM = pyproj.Proj('+proj=tmerc +lat_0=0 +lon_0=173 +k=0.9996 +x_0=1600000 +y_0=10000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs')

model = read_HYSPLIT_netCDF(FILENAME)

# project to NZTM
src_E, src_N = pyproj.transform(WGS84, NZTM, model.attrs['volcano_location'][1], model.attrs['volcano_location'][0])
model_E, model_N = pyproj.transform(WGS84, NZTM, model['lon'].values, model['lat'].values)

time_slice = model['total_deposition'].values[TIME_STEP, :, :] # grab time step from the gridded data        
np.place(time_slice, time_slice==0, np.nan) # replace all zero values with nan's

# tweak figure settings for readability
plt.figure(figsize=(14, 8))
mpl.rcParams.update({'font.size': 14})

# PLOT (projected) background geography
features = grab_GMT_features(-48, -33, 165, 180, RES) # this includes all of NZ
for f_lat, f_lon in zip(features['latitude'], features['longitude']):
    f_E, f_N = pyproj.transform(WGS84, NZTM, f_lon, f_lat)
    plt.plot(f_E, f_N, 'k', linewidth=0.75)

# PLOT ashfall (using log scale)
plt.pcolor(model_E, model_N, time_slice, cmap='hot_r', norm=colors.LogNorm(), vmin=ASH_MIN, vmax=ASH_MAX)
cb = plt.colorbar()
#plt.contour(model_E, model_N, time_slice, np.logspace(np.log10(ASH_MIN), np.log10(ASH_MAX), NUM_CON),\
#            colors='k', alpha = 0.5, linewidths=0.75)

# PLOT source location
plt.scatter(src_E, src_N, s=200, c='c', marker='*', edgecolors='k', linewidths=0.75, label='source')

# HARD-CODED plot limits for now
plt.xlim(1600000, 2100000)
plt.ylim(5600000, 6000000)
plt.gca().set_aspect('equal', adjustable='box')

plt.xlabel('easting')
plt.ylabel('northing')
plt.title(FILENAME.split('/')[-1] + '\n' + str((TIME_STEP+1)*3) + ' hrs post-event')
cb.set_label('ash thickness (mm)')

plt.legend();